## Render generated `.obj` mesh files as one interactive 3D image

In [5]:
# Import necessary libraries
import nibabel as nib
import numpy as np
from skimage import measure
import pyvista as pv
from matplotlib import cm
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap
from scipy.ndimage import gaussian_filter
import os
from glob import glob

In [2]:
# Filter subcortex
subcortex_file = 'Brainnetome_1mm_subcortex.nii.gz'
subcortex_file_L = 'Brainnetome_1mm_subcortex_L.nii.gz'
subcortex_file_R = 'Brainnetome_1mm_subcortex_R.nii.gz'

if not os.path.isfile(subcortex_file):

    # Set your input and output file paths
    input_file = 'BN_Atlas_246_1mm.nii.gz'

    # Load the NIfTI file
    img = nib.load(input_file)
    data = img.get_fdata()

    # Create a mask for the desired range (inclusive)
    mask = (data >= 211)

    # For left hemisphere, take odd numbers between 345 and 384 inclusive
    left_mask = (data >= 211) & (data % 2 == 1)
    # For right hemisphere, take even numbers between 345 and 384 inclusive
    right_mask = (data >= 211) & (data % 2 == 0)

    # Set values outside the range to 0, keep values inside unchanged
    filtered_data = np.where(mask, data, 0)
    filtered_data_L = np.where(left_mask, data, 0)
    filtered_data_R = np.where(right_mask, data, 0)

    # Save the filtered data as a new NIfTI file
    filtered_img = nib.Nifti1Image(filtered_data, img.affine, img.header)
    filtered_img_L = nib.Nifti1Image(filtered_data_L, img.affine, img.header)
    filtered_img_R = nib.Nifti1Image(filtered_data_R, img.affine, img.header)

    # Save the filtered data as a new NIfTI file
    nib.save(filtered_img, subcortex_file)
    print(f"Filtered both hemispheres saved as {subcortex_file}")

    nib.save(filtered_img_L, subcortex_file_L)
    print(f"Filtered left hemisphere saved as {subcortex_file_L}")

    nib.save(filtered_img_R, subcortex_file_R)
    print(f"Filtered right hemisphere saved as {subcortex_file_R}")


Filtered both hemispheres saved as Brainnetome_1mm_subcortex.nii.gz
Filtered left hemisphere saved as Brainnetome_1mm_subcortex_L.nii.gz
Filtered right hemisphere saved as Brainnetome_1mm_subcortex_R.nii.gz


In [3]:
%%bash 

source ~/.bashrc 
nii2mesh Brainnetome_1mm_subcortex_L.nii.gz -b 1 -l 0 -q 2 -r 0.01 -s 10 -a 1 individual_meshes/Brainnetome_1mm_subcortex_L.obj 

1/245
Skipping 1: no voxels with this intensity
2/245
Skipping 2: no voxels with this intensity
3/245
Skipping 3: no voxels with this intensity
4/245
Skipping 4: no voxels with this intensity
5/245
Skipping 5: no voxels with this intensity
6/245
Skipping 6: no voxels with this intensity
7/245
Skipping 7: no voxels with this intensity
8/245
Skipping 8: no voxels with this intensity
9/245
Skipping 9: no voxels with this intensity
10/245
Skipping 10: no voxels with this intensity
11/245
Skipping 11: no voxels with this intensity
12/245
Skipping 12: no voxels with this intensity
13/245
Skipping 13: no voxels with this intensity
14/245
Skipping 14: no voxels with this intensity
15/245
Skipping 15: no voxels with this intensity
16/245
Skipping 16: no voxels with this intensity
17/245
Skipping 17: no voxels with this intensity
18/245
Skipping 18: no voxels with this intensity
19/245
Skipping 19: no voxels with this intensity
20/245
Skipping 20: no voxels with this intensity
21/245
Skipping 21

We'll read in the meshes created by the `nii2mesh` program and combine them into one figure.

Note: this code will exclusively render the left hemisphere; if you want to keep both, uncomment line 8 instead of line 7.

In [4]:
# Path to the folder containing the .obj files
obj_folder = "individual_meshes"

# Find the .obj files in obj_folder
obj_files = glob(os.path.join(obj_folder, "*.obj"))

num_regions = int(len(obj_files)/2)  # Left hemisphere only
# num_regions = int(len(obj_files)) # Uncomment this line to include both hemispheres

# Define the file base for the .obj files
file_base = f"{obj_folder}/Brainnetome_1mm_subcortex_L"

# List to store meshes
meshes = []

# Generate num_regions colors from the 'plasma' colormap
# You can swap 'plasma' with any other colormap available in matplotlib
# cmap = cm.get_cmap('plasma', num_regions)
cmap = cm.get_cmap('gist_rainbow', num_regions)

# I want odd numbers from 345 to 384
region_indices = np.arange(211, 246, 2)

# Shuffle the region indices for random color assignment
np.random.seed(27)
shuffled_indices = region_indices.copy()
np.random.shuffle(shuffled_indices)

# Create a mapping from original region index to random color index
index_to_color_idx = {region: i for i, region in enumerate(shuffled_indices)}

# Iterate over each .obj file and add it to the meshes list
for i in region_indices:
    file_path = f"{file_base}{i}.obj"

    # Read the .obj file into a mesh
    mesh = pv.read(file_path)

    # Assign a scalar index corresponding to a shuffled color
    color_idx = index_to_color_idx[i]
    mesh.point_data['colors'] = np.full(mesh.n_points, color_idx)
    
    meshes.append(mesh)

# Combine all meshes into a single one
combined_mesh = meshes[0]
for mesh in meshes[1:]:
    combined_mesh += mesh

# Visualize the combined mesh
plotter = pv.Plotter()
# Add the mesh with custom colors (map the region index to region_colors)
plotter.add_mesh(combined_mesh, scalars="colors", cmap=cmap, show_edges=False)

plotter.show()

/var/folders/rn/z0683sns3tz7b_k0cbf8ft8w0000gq/T/ipykernel_63633/141794675.py:19: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('gist_rainbow', num_regions)


Widget(value='<iframe src="http://localhost:51184/index.html?ui=P_0x11f35b790_0&reconnect=auto" class="pyvista…

 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions
 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions
 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions
 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions
 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions
 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions
 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions
 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions
 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions
 JS Error => error: Uncaught TypeError: Cannot mix BigInt and other types, use explicit conversions


In [21]:
# Define which region to highlight
highlight_index = 245

# List to store meshes
meshes = []

# Define a custom 2-color colormap: gray60 and bright red (or any highlight color)
custom_cmap = ListedColormap(["lightgray", "crimson"])  # gray for background, crimson for highlighted region

# Iterate over each .obj file and assign color index
for i in region_indices:
    file_path = f"{file_base}{i}.obj"

    # Read the .obj file into a mesh
    mesh = pv.read(file_path)

    # Set color index: 1 for highlight, 0 for others
    color_idx = 1 if i == highlight_index else 0
    mesh.point_data['colors'] = np.full(mesh.n_points, color_idx)

    meshes.append(mesh)

# Combine all meshes into a single one
combined_mesh = meshes[0]
for mesh in meshes[1:]:
    combined_mesh += mesh

# Visualize the combined mesh
plotter = pv.Plotter()
plotter.add_mesh(combined_mesh, scalars="colors", cmap=custom_cmap, show_edges=False)
plotter.show()


Widget(value='<iframe src="http://localhost:51184/index.html?ui=P_0x17e67c910_16&reconnect=auto" class="pyvist…